In [1]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import pandas as pd
import json
from pandas import json_normalize
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from geoband.API import *
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from geopandas import GeoDataFrame
# GetCompasData로 주피터 노트북 경로에 파일 저장
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string

In [2]:
# 수원시 격자정보
suwon_112_grid = gpd.read_file('2.수원시_112신고(격자별 신고횟수).geojson').drop(["rep_cnt"],axis=1)

#### 링크 데이터 가공

In [94]:
링크 = gpd.read_file("링크.shp")

In [127]:
링크["FIRST_GU"] = 링크["FIRST_GU"].astype(str)

링크_4개시도현황 = 링크[(링크["FIRST_GU"].str[:4] == "3811")|
              (링크["FIRST_GU"].str[:4] == "3101")|
              (링크["FIRST_GU"].str[:4] == "3803")|
              (링크["FIRST_GU"].str[:2] == "11")]
링크_4개시도현황 = 링크_4개시도현황[["LANES","MAX_SPD","FIRST_GU","geometry"]].to_crs(epsg="4326")
링크_4개시도현황.columns = ["차로수","최고제한속도","시군구","geometry"]

In [166]:
수원시_링크현황 =  gpd.sjoin(링크_4개시도현황,suwon_112_grid, how="right",op="intersects").drop(["geometry","index_left","시군구"],axis=1)
# 최고제한속도는 min
# 차로수는 sum
# 시군구 drop
수원시_링크현황 = 수원시_링크현황.fillna(0)
수원시_링크현황.head(2)
수원시_링크현황_groupby = 수원시_링크현황.groupby(["gid"]).agg({"차로수":"sum","최고제한속도":"min"}).reset_index()

In [167]:
수원시_링크현황_groupby.to_csv("수원시_링크현황.csv",encoding="utf-8-sig",index=False)

#### 노드 데이터 가공

In [10]:
노드 = gpd.read_file("노드.shp")
노드 = 노드.to_crs(epsg="4326")
# NODE_TYPE : 101이 교차로
교차로_노드 = 노드[노드["NODE_TYPE"] == 101]
수원_교차로노드 = 교차로_노드[교차로_노드["DIST_ID2"].str[:4] == "3101"].reset_index(drop=True)
수원_교차로노드["lat"] = 수원_교차로노드["geometry"].apply(lambda point : point.y)
수원_교차로노드["lon"] = 수원_교차로노드["geometry"].apply(lambda point : point.x)
수원_교차로노드.drop(["X","Y"],axis=1,inplace=True)
수원_교차로노드 = 수원_교차로노드[["DIST_ID2","APPROCHES","geometry","lat","lon"]]
수원_교차로노드.columns = ["시군구","연결링크수","geometry","lat","lon"]
수원_교차로노드.head(2)

,시군구,연결링크수,geometry,lat,lon
0,31011,3,POINT (126.98185 37.30622),37.306219,126.981847
1,31011,3,POINT (126.97636 37.30571),37.305712,126.976363


In [14]:
수원_교차로노드.drop(["연결링크수","geometry"],axis=1).to_csv("수원시_교차로현황.csv",encoding="utf-8-sig",index=False)
# 말그대로 교차로 개수

수원_삼거리개수 = 수원_교차로노드[수원_교차로노드["연결링크수"] == 3]
수원_삼거리개수.drop(["geometry"],axis=1).to_csv("수원시_삼거리현황.csv",encoding="utf-8-sig",index=False)
# 삼거리 개수

수원_사거리개수 = 수원_교차로노드[수원_교차로노드["연결링크수"] == 4]
수원_사거리개수.drop(["geometry"],axis=1).to_csv("수원시_사거리현황.csv",encoding="utf-8-sig",index=False)
# 사거리 개수

수원_오거리개수 = 수원_교차로노드[수원_교차로노드["연결링크수"] > 4]
수원_오거리개수.drop(["geometry"],axis=1).to_csv("수원시_오거리현황.csv",encoding="utf-8-sig",index=False)
# 사거리 개수

교차로노드_4개시도현황 = 교차로_노드[(교차로_노드["DIST_ID2"].str[:4] == "3101")|
                     (교차로_노드["DIST_ID2"].str[:4] == "3811")|
                     (교차로_노드["DIST_ID2"].str[:4] == "3803")|
                     (교차로_노드["DIST_ID2"].str[:2] == "11")]
교차로노드_4개시도현황["lat"] = 교차로노드_4개시도현황["geometry"].apply(lambda point : point.y)
교차로노드_4개시도현황["lon"] = 교차로노드_4개시도현황["geometry"].apply(lambda point : point.x)
교차로노드_4개시도현황.drop(["X","Y"],axis=1,inplace=True)
교차로노드_4개시도현황[["DIST_ID2","APPROCHES","geometry","lat","lon"]].to_file("4개시도_교차로현황.geojson",driver="GeoJSON")